In [18]:
# sentiment analysis of tweets from twitter

# Import librairies

import tweepy
from tweepy import Cursor
import csv
import pandas as pd
import numpy as np
import re
from textblob import TextBlob
import matplotlib.pyplot as plt


from keys import *


In [7]:
# create the authentication object

authenticate = tweepy.OAuthHandler(API_KEY, API_SECRET_KEY)

# set access token and access token secret
authenticate.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

# create the API object
api = tweepy.API(authenticate, wait_on_rate_limit = True)

In [20]:
# extract 250 tweets from a twitter user
user = 'Enter Twitter Username'
domain = 'Enter Domain Politics/Sports'
posts = api.user_timeline(screen_name = user, count = 250, lang = "en", tweet_mode="extended")

# print the last 5 tweets from the user's account

print(f"Showing 5 recent tweets from {user}: \n")
i = 1
for tweet in posts[0:5]:
    print(str(i) +') ' + tweet.full_text + '\n')
    i = i+1

Showing 5 recent tweets from TheNotoriousMMA: 

1) RT @FocusedConor: @TheNotoriousMMA This is power. https://t.co/Ku5uqVPouc

2) Name the fights, the weights, the sports... https://t.co/jIRjaPu1Ry

3) 8 ko/tko across 30 fights = zero power. 
Don’t forget it teddy bear.

4) RT @TheFaceOfFight: @biggest_charms @TheNotoriousMMA No motherfookin doubt https://t.co/gTVzc9dgq5

5) RT @biggest_charms: Conor litteraly flew to khabib's country in Russia and recreated a scene from a rocky movie. We don't deserve this guy…



In [9]:
# Create a dataframe to hold the tweets
df = pd.DataFrame( [tweet.full_text for tweet in posts], columns=['Tweets'])

# show the first 5 rows of data
df.head()

,Tweets
0,RT @FocusedConor: @TheNotoriousMMA This is pow...
1,"Name the fights, the weights, the sports... ht..."
2,8 ko/tko across 30 fights = zero power. \nDon’...
3,RT @TheFaceOfFight: @biggest_charms @TheNotori...
4,RT @biggest_charms: Conor litteraly flew to kh...


In [10]:

# Function to clean the text

def cleanText(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text) # removes @mentions
    text = re.sub(r'#', '', text) # remove # symbol
    text = re.sub(r'/', '', text) # remove / symbol
    text = re.sub(r'-', '', text) # remove - symbol
    text = re.sub(r'"', '', text) # remove " symbol
    text = re.sub(r'[?|$|.|!|@|"|:|-|_|*]', '', text)
    text = re.sub(r'RT[\s]+', '', text) # remove RT
    text = re.sub(r'https?:\/\/\S+', '', text) # remove hyperlink
    
    return text

df['Tweets'] = df['Tweets'].apply(cleanText)


# show cleaned texted 
df

,Tweets
0,This is power httpstcoKu5uqVPouc
1,"Name the fights, the weights, the sports https..."
2,8 kotko across 30 fights = zero power \nDon’t ...
3,charms No motherfookin doubt httpstcogTVzc9dgq5
4,charms Conor litteraly flew to khabib's countr...
...,...
193,"Looking good Philip, my man Thank you for the ..."
194,Hahaha yes he did An excellent comeback this w...
195,Can’t quite put my finger on that one to be ho...
196,Excitement as well as supreme confidence \nIf ...


In [11]:
# create a function to get the subjectivity
def getSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity

# create a function to get the polarity
def getPolarity(text):
    return TextBlob(text).sentiment.polarity

# create columns in dataframe
df['Subjectivity'] = df['Tweets'].apply(getSubjectivity)
df['Polarity'] = df['Tweets'].apply(getPolarity)

# show new columns in dataframe

df



,Tweets,Subjectivity,Polarity
0,This is power httpstcoKu5uqVPouc,0.000000,0.000000
1,"Name the fights, the weights, the sports https...",0.000000,0.000000
2,8 kotko across 30 fights = zero power \nDon’t ...,0.000000,0.000000
3,charms No motherfookin doubt httpstcogTVzc9dgq5,0.000000,0.000000
4,charms Conor litteraly flew to khabib's countr...,0.000000,0.000000
...,...,...,...
193,"Looking good Philip, my man Thank you for the ...",0.600000,0.700000
194,Hahaha yes he did An excellent comeback this w...,0.540000,0.100000
195,Can’t quite put my finger on that one to be ho...,0.705102,0.397959
196,Excitement as well as supreme confidence \nIf ...,0.333333,0.250000


In [13]:
# create function to determine positive, neutral or negative

def getAnalysis(score):
    if score < 0:
        return 'Negative'
    elif score == 0:
        return 'Neutral'
    else:
        return 'Positive'

In [14]:
df['Analysis'] = df['Polarity'].apply(getAnalysis)

# show the dataframe
df

,Tweets,Subjectivity,Polarity,Analysis
0,This is power httpstcoKu5uqVPouc,0.000000,0.000000,Neutral
1,"Name the fights, the weights, the sports https...",0.000000,0.000000,Neutral
2,8 kotko across 30 fights = zero power \nDon’t ...,0.000000,0.000000,Neutral
3,charms No motherfookin doubt httpstcogTVzc9dgq5,0.000000,0.000000,Neutral
4,charms Conor litteraly flew to khabib's countr...,0.000000,0.000000,Neutral
...,...,...,...,...
193,"Looking good Philip, my man Thank you for the ...",0.600000,0.700000,Positive
194,Hahaha yes he did An excellent comeback this w...,0.540000,0.100000,Positive
195,Can’t quite put my finger on that one to be ho...,0.705102,0.397959,Positive
196,Excitement as well as supreme confidence \nIf ...,0.333333,0.250000,Positive


In [17]:
# write to csv file
data = df['Tweets']
Subjectivity = df['Subjectivity']
polarity = df['Polarity']
analysis = df['Analysis']

df['User'] = user
twitterUser = df['User']
df['Domain'] = domain
twitterDomain = df['Domain']

# write to csv file, ValueError error if not enough tweets        
with open('twitterData.csv', 'a', encoding="utf-8", newline='')  as f:
    writer = csv.writer(f)
    for w in range(100):
        writer.writerow([data[w],Subjectivity[w],polarity[w], analysis[w], twitterUser[w], twitterDomain[w]])
    


KeyError: 198